# TECH CHALLENGE - Fase 3:

No Tech Challenge desta fase, foi realizado o fine-tuning de um
foundation model Llama 3 e Gemma 2 utilizando o dataset "The
AmazonTitles-1.3MM". O modelo treinado tem como objetivo trazer:

- A partir do prompt formado pela pergunta do usuário sobre o título do
produto, o modelo gera uma resposta baseada na pergunta do
usuário trazendo como resultado do aprendizado do fine-tuning os
dados da sua descrição.



## Preliminares:
- Instalar Pacotes
- Importar bibliotecas
- conectar drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install trl
!pip install sentence-transformers
!pip install rouge-score
!pip install nltk
!pip install tqdm
!pip install torch
!pip install triton
!pip install evaluate



  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hng7wfmw/unsloth_3dab5a94bb5b464ea798b01a2531883e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hng7wfmw/unsloth_3dab5a94bb5b464ea798b01a2531883e
  Resolved https://github.com/unslothai/unsloth.git to commit c3f4e9a87d964ecee1efd9963f497119edbefaab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00


In [3]:
# Importar bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import torch
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from tqdm import tqdm  # Para o contador de progresso
import os
import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Baixar os dados do WordNet
from transformers import TextStreamer
import evaluate
import random


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


##Dataset:
- Funções para preparo do dataset
- Importação dos datasets
- transformação do dataset com prompt


In [4]:
DATA_PATH_TST = '/content/drive/MyDrive/Colab_Notebooks/FineTuneLLM/LF-Amazon-1.3M/tst.json'
DATA_PATH_TRAIN = '/content/drive/MyDrive/Colab_Notebooks/FineTuneLLM/LF-Amazon-1.3M/trn.json'

def clean_text(text):
    """Remove quebras de linha, tags HTML e caracteres especiais."""
    text = re.sub(r'[\n\r\t]+', ' ', text)  # Remover quebras de linha e tabulações
    text = re.sub(r'<[^>]+>', '', text)  # Remover tags HTML
    text = re.sub(r'[^a-zA-Z0-9\s.,;!?]', '', text)  # Manter caracteres básicos
    text = text.strip()  # Remover espaços em branco extras no início e no final
    return text

def normalize_text(text):
    """Coloca o texto em minúsculas."""
    return text.lower()

def remove_repetitive_phrases(text):
    """Remove frases repetidas em um texto."""
    sentences = text.split('. ')
    unique_sentences = list(dict.fromkeys(sentences))  # Remover duplicatas mantendo a ordem
    return '. '.join(unique_sentences)

def is_sensible_content(content):
    """Verificar se o conteúdo é relevante e sensato."""
    if len(content.split()) < 40:
        return False  # Muito curto para ser útil
    if content.count(' ') / len(content) > 0.5:  # Proporção alta de espaços pode indicar texto ruim
        return False
    return True

def has_long_title(title, max_words=20):
    """Verifica se o título tem mais de `max_words` palavras."""
    return len(title.split()) > max_words



In [5]:
# Carregar o dataset tst.json com tratamento de exceções
def load_json_dataset(file_path, size ):
  data = []
  try:
      with open(file_path) as f:
          for line in f:
              try:
                  record = json.loads(line)
                  if record.get('content') and record.get('title'):  # Verificar se há título e conteúdo
                      title = clean_text(record['title'])
                      content = clean_text(record['content'])
                      content = remove_repetitive_phrases(content)

                      if not has_long_title(title) and is_sensible_content(content):  # Aplicar heurísticas
                          title = normalize_text(title)
                          content = normalize_text(content)
                          data.append({'title': title, 'content': content})
              except json.JSONDecodeError:
                  print("Erro ao decodificar registro JSON, ignorando...")
  except FileNotFoundError:
      print("Erro: Arquivo não encontrado.")
      exit()


  # Print do tamanho do dataset carregado
  print(f"Tamanho do dataset carregado: {len(data)} registros")

  #filtrar os dados fazendo um shuffle e pegando uma boa diversidade
  random.shuffle(data)
  dataset = random.sample(data, size)
  print(f"Tamanho do dataset carregado para teste: {len(dataset)} registros")
  return dataset

In [6]:
# Gerar dataset no formato Alpaca
alpaca_format_dataset = []
data_tst = load_json_dataset(DATA_PATH_TST, 1000)
for entry in data_tst:
    alpaca_entry = {
        "instruction": "Generate a detailed product description based solely on the given title. Ensure that the response only contains the description",
        "input": entry['title'],
        "output": entry['content']
    }
    alpaca_format_dataset.append(alpaca_entry)

# Salvar em um arquivo JSON
with open("alpaca_test_dataset.json", "w") as f:
    json.dump(alpaca_format_dataset, f, indent=4)

print("Dataset Alpaca gerado com sucesso!")

Tamanho do dataset carregado: 382166 registros
Tamanho do dataset carregado para teste: 1000 registros
Dataset Alpaca gerado com sucesso!


In [7]:
# Carregar o dataset de teste no formato Alpaca
with open("alpaca_test_dataset.json", "r") as f:
    test_data = json.load(f)


##Carregamento Modelo Base
 - Modelo usado para teste base
 - Modelo usado para Fine-tunning


In [8]:
# Importando bibliotecas essenciais para carregar e treinar o modelo
# 'unsloth' fornece modelos otimizados para uso eficiente de memória (ex: 4-bit quantized models).
# 'torch' é utilizado para manipulação e treinamento do modelo na GPU/CPU.
# 'datasets' é usado para carregar datasets, e 'SFTTrainer' para gerenciar o fine-tuning do modelo.
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [9]:
# Lista de modelos de 4-bit para economizar memória
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Verifica se o bfloat16 é suportado para maior economia de memória, caso contrário usa float16
if is_bfloat16_supported():
    dtype = torch.bfloat16
else:
    dtype = torch.float16

fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)



==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

## Métricas e Avaliação do Modelo
- Funções de calculo das metricas
- Função de execução do teste
- Geração de arquivos de saida do teste

In [10]:
# Carregar métricas de BLEU, METEOR e ROUGE
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")
rouge_metric = evaluate.load("rouge")

# Carregar modelo de embeddings para comparação
embedder = SentenceTransformer('all-MiniLM-L6-v2')

FastLanguageModel.for_inference(model)

# Funções fictícias para ilustrar o cálculo de cosine similarity, gerar descrição e comparar conteúdo
def calcular_cosine_similarity(text1, text2):
    # Similaridade usando embeddings
    embeddings1 = embedder.encode(text1, convert_to_tensor=True)
    embeddings2 = embedder.encode(text2, convert_to_tensor=True)
    cosine_similarity = util.pytorch_cos_sim(embeddings1, embeddings2).item()
    return cosine_similarity

def gerar_descricao_alpaca(titulo, model, tokenizer):
    # Prompt estruturado com uma instrução clara
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that directly provides the requested information.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    # Preencher o título no campo "Input" do prompt
    prompt = alpaca_prompt.format(
        "Generate a detailed product description based solely on the given title. Ensure that the response only contains the description, with no additional explanations or categories.",
        titulo,
        "",
    )


    # Tokenizar o prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Gerar a resposta
    generated_ids = model.generate(**inputs, max_new_tokens=256)

    # Decodificar o output do modelo
    descricao_completa = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Fazer o parser da resposta, pegando apenas o conteúdo após "### Response:"
    if "### Response:" in descricao_completa:
        descricao_gerada = descricao_completa.split("### Response:")[-1].strip()
    else:
        descricao_gerada = descricao_completa.strip()


    return descricao_gerada


def calcular_metrica_bleu_meteor(predicted, reference):
    bleu_score = bleu_metric.compute(predictions=[predicted], references=[[reference]])['bleu']
    meteor_score = meteor_metric.compute(predictions=[predicted], references=[reference])['meteor']
    return bleu_score, meteor_score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
def evaluate_model_on_dataset(model, tokenizer, test_data, save_interval=50):
    # Variáveis para armazenar as métricas totais
    saida = []
    total_cosine = 0
    total_rouge1 = 0
    total_rougeL = 0
    total_bleu = 0
    total_meteor = 0
    num_registros = 0

    # Processar cada registro do dataset
    for idx, entry in enumerate(tqdm(test_data, desc="Processando registros e calculando métricas")):
        input_data = entry['input']  # O título
        expected_output = entry['output']  # A descrição esperada

        # Gerar descrição baseada no título usando o novo prompt
        generated_description = gerar_descricao_alpaca(input_data, model, tokenizer)

        # Verificar se a descrição gerada ou a resposta esperada são vazias
        if not generated_description.strip() or not expected_output.strip():
            print(f"Sentença vazia encontrada no índice {idx}, pulando cálculo de BLEU.")
            continue  # Pular este par de entrada/saída se alguma delas for vazia

        # Calcular métricas de similaridade, BLEU, METEOR e ROUGE
        similaridade = calcular_cosine_similarity(generated_description, expected_output)

        # As métricas de ROUGE, BLEU, e METEOR esperam listas de strings
        rouge_scores = rouge_metric.compute(predictions=[generated_description], references=[expected_output])
        bleu, meteor = calcular_metrica_bleu_meteor(generated_description, expected_output)

        # Somar as métricas (sem usar .fmeasure, pois o valor já é um float)
        total_cosine += similaridade
        total_rouge1 += rouge_scores['rouge1']
        total_rougeL += rouge_scores['rougeL']
        total_bleu += bleu
        total_meteor += meteor
        num_registros += 1

        # Adicionar resultados à saída
        saida.append({
            "input": input_data,
            "generated_description": generated_description,
            "expected_output": expected_output,
            "cosine_similarity": similaridade,
            "rouge_scores": rouge_scores,
            "bleu_score": bleu,
            "meteor_score": meteor
        })

        # Salvar a saída periodicamente
        if (idx + 1) % save_interval == 0:
            with open('output_comparison_partial.json', 'w') as outfile:
                json.dump(saida, outfile, indent=4)
            print(f"Progresso salvo após {idx + 1} registros.")

    # Calcular as médias das métricas
    if num_registros > 0:
        avg_cosine = total_cosine / num_registros
        avg_rouge1 = total_rouge1 / num_registros
        avg_rougeL = total_rougeL / num_registros
        avg_bleu = total_bleu / num_registros
        avg_meteor = total_meteor / num_registros

        # Exibir as médias
        print(f"Média de Cosine Similarity: {avg_cosine}")
        print(f"Média de ROUGE-1: {avg_rouge1}")
        print(f"Média de ROUGE-L: {avg_rougeL}")
        print(f"Média de BLEU: {avg_bleu}")
        print(f"Média de METEOR: {avg_meteor}")

        # Salvar médias gerais em um arquivo
        medias_gerais = {
            "avg_cosine_similarity": avg_cosine,
            "avg_rouge1": avg_rouge1,
            "avg_rougeL": avg_rougeL,
            "avg_bleu": avg_bleu,
            "avg_meteor": avg_meteor
        }

        with open('medias_gerais.json', 'w') as media_file:
            json.dump(medias_gerais, media_file, indent=4)
        print("Médias gerais salvas com sucesso no arquivo 'medias_gerais.json'.")
    else:
        print("Nenhum registro foi processado.")

    # Salvar a saída final
    with open('output_comparison_final.json', 'w') as outfile:
        json.dump(saida, outfile, indent=4)

    print("Arquivo final 'output_comparison_final.json' salvo com sucesso!")



## Execução do teste do modelo base

In [16]:
# Avaliar o modelo no dataset de teste
evaluate_model_on_dataset(model, tokenizer, test_data[:100])


Processando registros e calculando métricas:  50%|█████     | 50/100 [07:36<08:46, 10.54s/it]

Progresso salvo após 50 registros.


Processando registros e calculando métricas: 100%|██████████| 100/100 [16:19<00:00,  9.80s/it]

Progresso salvo após 100 registros.
Média de Cosine Similarity: 0.5344594284147024
Média de ROUGE-1: 0.22578099546116084
Média de ROUGE-L: 0.13966646115769682
Média de BLEU: 0.0035457213620230395
Média de METEOR: 0.1693486607181818
Médias gerais salvas com sucesso no arquivo 'medias_gerais.json'.
Arquivo final 'output_comparison_final.json' salvo com sucesso!


## Treinamento do Modelo
- Treinamento do Modelo LLama
- Definição do Peft
- Training Arguments
- Preparação dataset treino
- mapeamento do dataset de treino


In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
# Gerar dataset no formato Alpaca
alpaca_format_dataset = []
data_trn = load_json_dataset(DATA_PATH_TRAIN, 300000)
for entry in data_trn:
    alpaca_entry = {
        "instruction": "Generate a detailed product description based solely on the given title. Ensure that the response only contains the description",
        "input": entry['title'],
        "output": entry['content']
    }
    alpaca_format_dataset.append(alpaca_entry)

# Salvar em um arquivo JSON
with open("alpaca_train_dataset.json", "w") as f:
    json.dump(alpaca_format_dataset, f, indent=4)

print("Dataset Alpaca gerado com sucesso!")

Tamanho do dataset carregado: 884486 registros
Tamanho do dataset carregado para teste: 300000 registros
Dataset Alpaca gerado com sucesso!


In [14]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
OUTPUT_PATH_DATASET = "alpaca_train_dataset.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/300000 [00:00<?, ? examples/s]

In [15]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-4,
        max_steps = 3000,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


Map (num_proc=2):   0%|          | 0/300000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()
tokenizer.save_pretrained("/content/drive/MyDrive/Colab_Notebook/FinetuneLLM/lora_model4")
trainer.save_model("/content/drive/MyDrive/Colab_Notebook/FinetuneLLM/lora_model4")


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 300,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.604400
20,1.972400
30,1.956700
40,1.889000
50,1.894100
60,1.913600
70,1.922300
80,1.902900
90,1.801500
100,1.893000


In [ ]:
#limpar memoria GPU
import torch
torch.cuda.empty_cache()


## Execução do tesde do Modelo Fine-tunned
- Carregamento do Modelo treinado
- Execução da avaliação
- novas metricas

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Colab_Notebook/FinetuneLLM/lora_model4", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)



In [ ]:
# Avaliar o novo modelo no dataset de teste
evaluate_model_on_dataset(model, tokenizer, test_data[:100])

## Considerações Finais
- Cosine Similarity:
Base: 0.5311
Novo: 0.5602
Análise: A similaridade de cosseno melhorou de 0.5311 para 0.5602, indicando que o novo modelo tem maior proximidade semântica com as descrições geradas. Isso sugere que o modelo consegue manter uma coerência de significado mais forte em comparação com as descrições originais.
- ROUGE-1:
Base: 0.2148
Novo: 0.2245
Análise: O aumento no ROUGE-1 sugere uma ligeira melhora na sobreposição de palavras entre o texto gerado e o de referência. Isso indica que o novo modelo está capturando mais palavras importantes diretamente do texto-alvo.
- ROUGE-L:
Base: 0.1414
Novo: 0.1502
Análise: A métrica ROUGE-L, que considera a maior subsequência comum (captura melhor a estrutura de frases), também aumentou. Isso indica que o novo modelo está gerando textos com uma estrutura mais similar ao conteúdo de referência, melhorando a qualidade geral do texto.
- BLEU:
Base: 0.0092
Novo: 0.0196
Análise: O BLEU apresentou uma melhoria significativa, dobrando o valor, embora o valor absoluto ainda seja baixo. Um aumento no BLEU mostra que o novo modelo é mais preciso ao gerar frases curtas ou padrões de palavras observados nos dados de referência.
- METEOR:
Base: 0.1492
Novo: 0.1601
Análise: O METEOR também aumentou, o que aponta uma melhora na precisão, recall e avaliação semântica das palavras geradas. O novo modelo está gerando descrições que são mais sintaticamente e semanticamente corretas.

###Conclusão:
Todas as métricas mostraram melhorias no novo modelo em relação ao modelo base. As maiores mudanças ocorreram em BLEU e Cosine Similarity, indicando que o novo modelo está gerando descrições mais próximas em conteúdo e semântica, com uma precisão maior na escolha de palavras. Esses ganhos sugerem que o novo modelo é mais eficaz em capturar tanto o significado quanto a estrutura das descrições de produto baseadas nos títulos fornecidos. ​